In [24]:
import pandas as pd
import numpy as np
df= pd.read_csv("train_merged_df.csv")
df_test=pd.read_csv("test_merged_df.csv")
df.head()


,datetime,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25,is_holiday,is_weekend,temperature_2m,...,wind_gusts_10m,vapour_pressure_deficit,cape,evapotranspiration,et0_fao_evapotranspiration,snowfall,rain,showers,weather_code,visibility
0,2020-01-01 00:00:00,2020-01-01 00,42.9,0.718,15.7,73.1,64.4,1,0,0.9,...,12.2,0.00,NaN,NaN,0.0,0.0,0.0,0.0,3,NaN
1,2020-01-01 01:00:00,2020-01-01 01,33.6,0.587,10.1,74.8,66.0,1,0,-0.1,...,5.8,0.00,NaN,NaN,0.0,0.0,0.0,0.0,3,NaN
2,2020-01-01 02:00:00,2020-01-01 02,29.3,NaN,5.1,51.0,44.9,1,0,2.6,...,7.6,0.01,NaN,NaN,0.0,0.0,0.0,0.0,3,NaN
3,2020-01-01 03:00:00,2020-01-01 03,30.5,0.246,7.2,27.7,25.1,1,0,2.1,...,7.2,0.00,NaN,NaN,0.0,0.0,0.0,0.0,3,NaN
4,2020-01-01 04:00:00,2020-01-01 04,29.3,0.204,8.3,15.3,13.6,1,0,1.9,...,8.6,0.00,NaN,NaN,0.0,0.0,0.0,0.0,3,NaN


In [25]:
lock = pd.read_csv("/Users/nandanasreeraj/air/Air_Quality_Prediction_Paris/data/france_lockdowns.csv")



In [26]:
# clean strings, then cast to datetimes (dayfirst!)
lock['Start'] = pd.to_datetime(lock['Start'].astype(str).str.strip(), dayfirst=True, errors='coerce')
lock['End']   = pd.to_datetime(lock['End'].astype(str).str.strip(),   dayfirst=True, errors='coerce')

# fill open-ended End with something within df range to avoid overflow
end_fill = getattr(df['datetime'], 'max', lambda: None)()
if pd.isna(end_fill):
    end_fill = pd.Timestamp('2262-04-11')  # pandas max
lock['End'] = lock['End'].fillna(end_fill)

# ensure ordering required by merge_asof
lock = lock.sort_values('Start')

In [27]:



df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce')
df = df.sort_values('datetime')

# --- 3) As-of merge on Start (keeps ALL df rows) ---
out = pd.merge_asof(
    df,
    lock,
    left_on='datetime',
    right_on='Start',
    direction='backward',
    allow_exact_matches=True
)





df_test['datetime'] = pd.to_datetime(df_test['datetime'], errors='coerce')
df_test = df_test.sort_values('datetime')

# --- 3) As-of merge on Start (keeps ALL df rows) ---
out_test = pd.merge_asof(
    df_test,
    lock,
    left_on='datetime',
    right_on='Start',
    direction='backward',
    allow_exact_matches=True
)
# --- 4) Invalidate rows where datetime is past the End of the matched period ---
# --- 5) Map to code (FULL=2, PARTIAL=1, else 0) ---

mask_outside = out['End'].notna() & (out['datetime'] > out['End'])
out.loc[mask_outside, ['Start', 'End', 'Type']] = np.nan

out['lockdown_code'] = out['Type'].map({'FULL': 2, 'PARTIAL': 1}).fillna(0).astype(int)


mask_outside_test = out_test['End'].notna() & (out_test['datetime'] > out_test['End'])
out_test.loc[mask_outside_test, ['Start', 'End', 'Type']] = np.nan

out_test['lockdown_code'] = out_test['Type'].map({'FULL': 2, 'PARTIAL': 1}).fillna(0).astype(int)

In [28]:
out = out.drop(columns=['Start', 'End', 'Type'])
out_test = out_test.drop(columns=['Start', 'End', 'Type'])



In [29]:
out.head()

,datetime,id,valeur_NO2,valeur_CO,valeur_O3,valeur_PM10,valeur_PM25,is_holiday,is_weekend,temperature_2m,...,vapour_pressure_deficit,cape,evapotranspiration,et0_fao_evapotranspiration,snowfall,rain,showers,weather_code,visibility,lockdown_code
0,2020-01-01 00:00:00,2020-01-01 00,42.9,0.718,15.7,73.1,64.4,1,0,0.9,...,0.00,NaN,NaN,0.0,0.0,0.0,0.0,3,NaN,0
1,2020-01-01 01:00:00,2020-01-01 01,33.6,0.587,10.1,74.8,66.0,1,0,-0.1,...,0.00,NaN,NaN,0.0,0.0,0.0,0.0,3,NaN,0
2,2020-01-01 02:00:00,2020-01-01 02,29.3,NaN,5.1,51.0,44.9,1,0,2.6,...,0.01,NaN,NaN,0.0,0.0,0.0,0.0,3,NaN,0
3,2020-01-01 03:00:00,2020-01-01 03,30.5,0.246,7.2,27.7,25.1,1,0,2.1,...,0.00,NaN,NaN,0.0,0.0,0.0,0.0,3,NaN,0
4,2020-01-01 04:00:00,2020-01-01 04,29.3,0.204,8.3,15.3,13.6,1,0,1.9,...,0.00,NaN,NaN,0.0,0.0,0.0,0.0,3,NaN,0


In [32]:
out_test["lockdown_code"].nunique()



1

In [33]:
out.to_csv("traindata_with_lockdown.csv")
out_test.to_csv("testdata_with_lockdown.csv")
